In [2]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv("spam.csv")
df = df.drop(df.loc[:, "char_freq_;" : "capital_run_length_total"], axis=1)
df

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_direct,word_freq_cs,word_freq_meeting,word_freq_original,word_freq_project,word_freq_re,word_freq_edu,word_freq_table,word_freq_conference,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.06,0.0,0.0,0.12,0.00,0.06,0.06,0.0,0.0,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.00,0.0,0.0,0.00,0.31,0.31,0.31,0.0,0.0,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.0,0.00,0.00,0.00,2.00,0.0,0.0,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.0,0.00,0.00,0.00,1.20,0.0,0.0,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.0,0.0,0.00,0.32,0.00,0.32,0.0,0.0,0


In [41]:
df.size

225449

In [51]:
def split_dataset(data, part_for_test=0.1):
    inds = np.random.randint(0, data.size, int(data.size * part_for_test))
    test = data.iloc[inds, :]
    train = data.drop(data.iloc[inds, :], axis=0)
    return test, train

In [52]:
split_dataset(df)

13 225439


IndexError: positional indexers are out-of-bounds